In [24]:
import os
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import numpy as np
import matplotlib.pyplot as plt
from termcolor import colored
from keras.utils import plot_model

In [2]:
from keras.models import load_model

In [3]:
#Aqui tienes q importar tu version
from lime import lime_tridimensional

In [4]:
#estos datos son para la red
# Initializing var
n_features = 8 
n_outputs = 14  # solo para LSTM y CNN sino  =1
lookback = 14
potencia = 3

#datos de entrenamientos
training_set = np.loadtxt(
            r"C:\\Users\\raudi\Desktop\\aqui\\red neuronal\\LIME\\Numpy_sets\\Numpy_training_setEstructurado_" + 'PARQ. SOLAR UNIV.  LAS VILLAS' + ".txt")
testing_set = np.loadtxt(
            r"C:\\Users\\raudi\Desktop\\aqui\\red neuronal\\LIME\\Numpy_sets\\Numpy_testing_setEstructurado_" + 'PARQ. SOLAR UNIV.  LAS VILLAS' + ".txt")

In [5]:
#Esto para la red
def create_windows (lookback,pos_potencia):  # lookback ventana hacia atras y posicion del atributor predictor
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    print(training_set)
    n_future = n_outputs # Number of days we want top predict into the future
    n_past = lookback     # Number of past days we want to use to predict the future
    
    #Train
    for i in range(n_past, len(training_set) - n_future +1):
        X_train.append(training_set[i - n_past:i, :])                         
        #y_train.append(training_set[i + n_future - 1:i + n_future,2])         #"1 salida Column potencia"
        y_train.append(training_set[i:i + n_future,pos_potencia].reshape(-1,1))         # Multiples salidas 
        

    #Test    
  
    for i in range(n_past, len(testing_set) - n_future +1,n_future):
        X_test.append(testing_set[i - n_past:i, :])                                  # modelo multivariado 
        #y_test.append(testing_set[i + n_future - 1:i + n_future,2])                 # la primera medicion del dia despues 
        y_test.append(testing_set[i:i + n_future,pos_potencia].reshape(-1,1))        # todas las predcciones del dia despues o 14 horas despues
        

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    print('X_train shape == {}.'.format(X_train.shape))
    print('y_train shape == {}.'.format(y_train.shape))

    print('X_test shape == {}.'.format(X_test.shape))
    print('y_test shape == {}.'.format(y_test.shape))

    return X_train, y_train, X_test, y_test    

In [6]:
#Se dividen en X_train, y_train, X_test, y_test
X_train, y_train, X_test, y_test = create_windows(lookback, potencia)
print("X_train: {}\ny_train {}\nX_test {}\ny_test{}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

[[ 0.00000000e+00  2.07000000e+01  2.01000000e+01 ... -2.22520934e-01
  -9.75741567e-01 -2.18925544e-01]
 [ 0.00000000e+00  2.00000000e+01  1.95000000e+01 ...  2.22520934e-01
  -9.76009840e-01 -2.17726418e-01]
 [ 1.98500000e+02  2.61000000e+01  2.26000000e+01 ...  6.23489802e-01
  -9.76276638e-01 -2.16526962e-01]
 ...
 [ 2.85800000e+02  2.61000000e+01  2.51000000e+01 ... -6.23489802e-01
  -4.20454504e-01  9.07313623e-01]
 [ 1.78300000e+02  2.88000000e+01  3.38000000e+01 ... -9.00968868e-01
  -4.19339307e-01  9.07829579e-01]
 [ 1.24800000e+02  2.65000000e+01  3.05000000e+01 ... -1.00000000e+00
  -4.18223477e-01  9.08344165e-01]]
X_train shape == (9571, 14, 8).
y_train shape == (9571, 14, 1).
X_test shape == (170, 14, 8).
y_test shape == (170, 14, 1).
X_train: (9571, 14, 8)
y_train (9571, 14, 1)
X_test (170, 14, 8)
y_test(170, 14, 1)


In [7]:
def CNN_model(lookback = lookback, 
              conv1_filters = 32, 
              conv1_kernel_size = 11,
              conv2_filters = 32, 
              conv2_kernel_size = 11,
              pool_size = 2,
              dropout = 0.5,
              dense_units = 100,
              activation = 'linear',n_outputs= n_outputs):
    model2 = Sequential()
    model2.add(Conv1D(filters= conv1_filters, kernel_size=conv1_kernel_size, activation='relu', input_shape=(14,8))) 
    model2.add(Conv1D(filters= conv2_filters, kernel_size=conv2_kernel_size, activation='relu'))
    model2.add(MaxPooling1D(pool_size=pool_size))
    model2.add(Dropout(dropout))
    model2.add(Flatten())
    model2.add(Dense(dense_units, activation=activation))
    model2.add(Dense(1))
    model2.compile(optimizer='adam', loss='mse')
    return model2 

In [8]:
model_CNN = CNN_model(
    lookback=lookback, 
    conv1_filters=8, 
    conv1_kernel_size=2, 
    conv2_filters=4, 
    conv2_kernel_size=2, 
    pool_size=2, 
    dropout=0.50,
    dense_units=100,
    activation='sigmoid')
#Leyendo los pesos
model_CNN.load_weights('C:\\Users\\raudi\\Desktop\\Practia Laboral Investigativa 3ro CC\\red neuronal\\LIME\\CNN_model_UCLV.h5')

c:\Users\raudi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [9]:
model_CNN.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 13, 8)             │        136 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv1d_1 (Conv1D)               │ (None, 12, 4)             │         68 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 6, 4)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 6, 4)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 24)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 100)               │      2,500 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 1)                 │        101 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,805 (10.96 KB)

 Trainable params: 2,805 (10.96 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Preparar datos
# Aplanar los datos para pasarlos al método
def aplanar(x):
    aplanado = x.reshape(x.shape[0], -1)
    return aplanado



In [11]:
feature_names_aux = ['Irradiancia', 'Tambiente', 'Tmodulo', 'Potencia', 'Day sin', 'Day cos', 'Year sin', 'Year cos']
feature_names = ['Day-{}-Feature-{}'.format(i, feature_names_aux[j]) for i in range(lookback) for j in range(n_features)]

In [12]:
print(feature_names)

['Day-0-Feature-Irradiancia', 'Day-0-Feature-Tambiente', 'Day-0-Feature-Tmodulo', 'Day-0-Feature-Potencia', 'Day-0-Feature-Day sin', 'Day-0-Feature-Day cos', 'Day-0-Feature-Year sin', 'Day-0-Feature-Year cos', 'Day-1-Feature-Irradiancia', 'Day-1-Feature-Tambiente', 'Day-1-Feature-Tmodulo', 'Day-1-Feature-Potencia', 'Day-1-Feature-Day sin', 'Day-1-Feature-Day cos', 'Day-1-Feature-Year sin', 'Day-1-Feature-Year cos', 'Day-2-Feature-Irradiancia', 'Day-2-Feature-Tambiente', 'Day-2-Feature-Tmodulo', 'Day-2-Feature-Potencia', 'Day-2-Feature-Day sin', 'Day-2-Feature-Day cos', 'Day-2-Feature-Year sin', 'Day-2-Feature-Year cos', 'Day-3-Feature-Irradiancia', 'Day-3-Feature-Tambiente', 'Day-3-Feature-Tmodulo', 'Day-3-Feature-Potencia', 'Day-3-Feature-Day sin', 'Day-3-Feature-Day cos', 'Day-3-Feature-Year sin', 'Day-3-Feature-Year cos', 'Day-4-Feature-Irradiancia', 'Day-4-Feature-Tambiente', 'Day-4-Feature-Tmodulo', 'Day-4-Feature-Potencia', 'Day-4-Feature-Day sin', 'Day-4-Feature-Day cos', 'Day-4

In [13]:
X_train_2d=aplanar(X_train)
X_train_2d.shape


(9571, 112)

In [15]:
# Crea un objeto Lime para explicar las predicciones
explainer = lime_tridimensional.LimeTabularExplainer(X_train_2d,
                                            feature_names=feature_names,
                                            class_names=['output'],
                                            discretize_continuous=False, mode="regression")

In [16]:
X_test_2d = aplanar(X_test)
X_test_2d.shape


(170, 112)

In [33]:
# Explica la predicción de tu modelo para la instancia seleccionada
explanation = explainer.explain_instance(X_test_2d[3], model_CNN.predict, num_features=len(feature_names)).as_list()

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [34]:

# Muestra las características más importantes que contribuyeron a la predicción
colored_explanation =[(colored(feature_names,'green'),weight)if weight>0 else (colored(feature_names,'red'),weight)
for feature_names ,weight in explanation]

for feature_names, weight in colored_explanation:
    print(f"{feature_names}: {weight}")


Day-6-Feature-Irradiancia: 0.008567599474536327
Day-5-Feature-Irradiancia: 0.0075825237313442
Day-9-Feature-Irradiancia: 0.006394705599575805
Day-10-Feature-Irradiancia: 0.006311235921622462
Day-2-Feature-Irradiancia: 0.005278470403161022
Day-7-Feature-Irradiancia: 0.004270142928693221
Day-3-Feature-Irradiancia: 0.0035924727446529233
Day-4-Feature-Irradiancia: 0.003524204656185193
Day-13-Feature-Irradiancia: 0.002956099661159932
Day-1-Feature-Irradiancia: 0.0026891446265475704
Day-11-Feature-Irradiancia: 0.00239050662922868
Day-0-Feature-Irradiancia: 0.0007940570426381116
Day-2-Feature-Day sin: -0.0006626958452370488
Day-8-Feature-Year cos: -0.0006398611966397559
Day-4-Feature-Year cos: -0.0004992144308268278
Day-9-Feature-Day cos: 0.0004724399393506048
Day-11-Feature-Year cos: -0.00046855041350706304
Day-1-Feature-Day sin: 0.0004421647106108154
Day-3-Feature-Potencia: 0.0004232201362076752
Day-3-Feature-Year cos: -0.0004190547827680637
Day-4-Feature-Potencia: -0.0004089788261813791
Da